# Economic rurality by state

[Leaflet.js docs](https://leafletjs.com/reference-1.7.1.html) - specification of `style` arguments. 

In [ ]:
import ipywidgets as widgets
import ipyleaflet as leaflet
import branca.colormap
import pandas as pd
import joblib
import json
import geopandas
import shapely
import matplotlib.pyplot as plt

from rurec import rurality
from rurec import geo_gazetteer
from rurec import ers_codes

In [ ]:
memory = joblib.Memory('../cache')

## prepare data

In [ ]:
data = {
    'CBSA': {},
    'UI': {},
    'RUC': {}
}

### rurality

In [ ]:
data['CBSA']['rural levels'] = {'M1': False, 'M2': False}
data['UI']['rural levels'] = {'1': False, '2': False, '3': False, '4': False, '5': False,
                              '6': True, '7': True, '8': True, '9': True, '10': True, '11': True, '12': True}
data['RUC']['rural levels'] = {'0': False, '1': False, '2': False, '3': False, '4': False, '6': False, '8': True,
                               '5': True, '7': True, '9': True}

### economics by year, geography, FAI and rurality

In [ ]:
years = [1997, 2007, 2017]
# states = ['WI', 'CT']
states = None
cols = ['YEAR', 'STATE', 'FIPS_CODE', 'EMPLOYEES', 'NAICS', 'UI_CODE', 'RUC_CODE', 'CBSA_LEVEL']
df = rurality.get_df(years=years, cols=cols, states=states)
df['STATE'] = df['STATE'].cat.remove_unused_categories()
df = df.rename(columns={'STATE': 'STATE_ABBR', 
                        'FIPS_CODE': 'STATECOUNTY_CODE'})
df['RURAL_CBSA'] = df['CBSA_LEVEL'].map(data['CBSA']['rural levels']).fillna(True)
df['RURAL_UI'] = df['UI_CODE'].map(data['UI']['rural levels'])
df['RURAL_RUC'] = df['RUC_CODE'].map(data['RUC']['rural levels'])

fai = json.load(open('../data/fai_subsectors.json'))
fai = [c for subsector in fai.values() for c in subsector.keys()]
fai = pd.DataFrame({'NAICS': fai})
fai['FAI'] = True
df['NAICS'] = df['NAICS'].str[:6]
df = df.merge(fai, 'left', 'NAICS')
df['FAI'] = df['FAI'].fillna(False)

for rural_classification in ['CBSA', 'UI', 'RUC']:
    rural_col = f'RURAL_{rural_classification}'
    d = df.groupby(['YEAR', 'STATE_ABBR', 'FAI', rural_col])['EMPLOYEES'].agg(['size', 'sum'])
    d.columns.name = 'SIZE_MEASURE'
    d = d.rename(columns={'size': 'establishments', 'sum': 'employees'})
    d = d.stack('SIZE_MEASURE').unstack([rural_col, 'FAI']).sort_index(1)
    d = d.apply(lambda c: c / d.sum(1))
    data[rural_classification]['econ shares'] = d

d = df.groupby(['YEAR', 'STATE_ABBR', 'STATECOUNTY_CODE', 'FAI'])['EMPLOYEES'].sum()
d = d.unstack('FAI')
d = d[True] / d.sum(1)
data['county FAI share'] = d

joblib.dump(data, '../cache/voila.pkl')

In [ ]:
data = joblib.load('../cache/voila.pkl')

## Geography and population

In [ ]:
@memory.cache
def download_cbsa_boundaries():
    return geopandas.read_file('https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_cbsa_20m.zip')

@memory.cache
def download_county_boundaries():
    return geopandas.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_20m.zip')


In [ ]:
def prep_cbsa():
    df = download_cbsa_boundaries()[['CBSAFP', 'NAME', 'LSAD', 'geometry']]
    df = df.rename(columns={'CBSAFP': 'CBSA_CODE', 'NAME': 'CBSA_NAME', 'LSAD': 'CBSA_LEVEL'})
    df['CBSA_LABEL'] = df['CBSA_LEVEL'].map({'M1': 'Metropolitan', 'M2': 'Micropolitan'})
    df['CBSA_COLORCODE'] = df['CBSA_LEVEL'].map({'M1': 1, 'M2': 0.5})
    df['RURAL_CBSA'] = df['CBSA_LEVEL'].map(data['CBSA']['rural levels']).fillna(True)
    return df


def prep_county():
    df = download_county_boundaries()
    df = df.drop(columns=['COUNTYNS', 'AFFGEOID', 'LSAD', 'ALAND', 'AWATER'])
    df = df.rename(columns={'STATEFP': 'STATE_CODE', 'COUNTYFP': 'COUNTY_CODE', 'GEOID': 'STATECOUNTY_CODE', 'NAME': 'COUNTY_NAME'})
    df['STATE_ABBR'] = df['STATE_CODE'].map(geo_gazetteer.get_mapping('STATE_CODE', 'STATE_ABBR'))
    df['STATE_NAME'] = df['STATE_CODE'].map(geo_gazetteer.get_mapping('STATE_CODE', 'STATE_NAME'))
    df0 = df
    
    df = ers_codes.get_ui_df()
    df = df.rename(columns={'FIPS': 'STATECOUNTY_CODE', 'UI_CODE': 'UI_LEVEL', 'UI_CODE_DESCRIPTION': 'UI_LABEL'})
    df = df[df['UI_YEAR'] == 2013]
    df = df[['STATECOUNTY_CODE', 'UI_LEVEL', 'UI_LABEL']]
    df['UI_COLORCODE'] = (df['UI_LEVEL'].astype(int) - 1) / 11
    df['RURAL_UI'] = df['UI_LEVEL'].map(data['UI']['rural levels'])
    df0 = df0.merge(df, 'left', 'STATECOUNTY_CODE')
    
    df = ers_codes.get_ruc_df()
    df = df.rename(columns={'FIPS': 'STATECOUNTY_CODE', 'RUC_CODE': 'RUC_LEVEL', 'RUC_CODE_DESCRIPTION': 'RUC_LABEL'})
    df = df[df['RUC_YEAR'] == 2013]
    df = df[['STATECOUNTY_CODE', 'RUC_LEVEL', 'RUC_LABEL']]
    df['RUC_COLORCODE'] = df['RUC_LEVEL'].astype(int) / 9
    df['RURAL_RUC'] = df['RUC_LEVEL'].map(data['RUC']['rural levels'])
    df0 = df0.merge(df, 'left', 'STATECOUNTY_CODE')
    
    return df0

data['CBSA']['rural areas'] = prep_cbsa()
data['UI']['rural areas'] = data['RUC']['rural areas'] = prep_county()

In [ ]:
def prep_county_fai():
    df = download_county_boundaries()[['geometry', 'GEOID']]
    df = df.rename(columns={'GEOID': 'STATECOUNTY_CODE'})

    d = data['county FAI share']
    d = d.to_frame('FAI_EMP_SHARE').reset_index().dropna(subset=['FAI_EMP_SHARE'])
    df = df.merge(d, 'right', 'STATECOUNTY_CODE')
    return df

data['county FAI share geo'] = prep_county_fai()

## dashboard

In [ ]:
class TableShares:
    def __init__(self):
        self.widget = widgets.Output()
    
    def update(self, state, rural_classification):
        idx = pd.IndexSlice
        df = data[rural_classification]['econ shares'].loc[idx[:, state, 'employees'], :].droplevel(['STATE_ABBR', 'SIZE_MEASURE'])
        with self.widget:
            self.widget.clear_output(True)
            print('Share of employment')
            display(df.style.format('{:.2%}'))
            
w_shares = TableShares()

In [ ]:
w_shares.widget

In [ ]:
class Map:
    def __init__(self):
        self.widget = m = leaflet.Map(center=(38, -95), zoom=4)
        m.layout.height = '500px'
        m.layout.width = '800px'
        self.legend = leaflet.LegendControl({}, position='topright')
        m.add_control(self.legend)
        self.layers_control = leaflet.LayersControl()
        m.add_control(self.layers_control)
        
        self.rural_classification = 'CBSA'
        self.rural_layer = l = leaflet.GeoJSON()
        m.add_layer(l)
        
        self.fai_layer = leaflet.GeoJSON()
        m.add_layer(self.fai_layer)
        
    def rural_style(self, area):
        cc = area['properties'][f'{self.rural_classification}_COLORCODE']
        fc = branca.colormap.linear.Reds_03.rgb_hex_str(cc)
        rural = area['properties'][f'RURAL_{self.rural_classification}']
        c = 'white' if rural else 'black'
        return dict(fillColor=fc, color=c)
    
    def replace_rural_layer(self, state, rural_classification):
        self.rural_classification = rural_classification
        df = data[rural_classification]['rural areas']
        if rural_classification == 'CBSA':
            df = df[df['CBSA_NAME'].str.contains(state)]
        else:
            df = df[df['STATE_ABBR'] == state]
            
        self.widget.remove_layer(self.rural_layer)
        self.rural_layer = l = leaflet.GeoJSON(name='Rurality', data=json.loads(df.to_json()), style_callback=self.rural_style)
        self.widget.add_layer(l)
        self.widget.fit_bounds(self.poly_bounds(df.geometry))
        
    def fai_style(self, area):
        cc = area['properties']['FAI_EMP_SHARE']
        cm = branca.colormap.linear.Blues_03.scale(vmax=0.1)
        fc = cm.rgb_hex_str(cc)
        return dict(fillColor=fc)
    
    def replace_fai_layer(self, state):
        df = data['county FAI share geo']
        df = df[(df['YEAR'] == 2017) & (df['STATE_ABBR'] == state)]
        self.widget.remove_layer(self.fai_layer)
        self.fai_layer = leaflet.GeoJSON(name='FAI', data=json.loads(df.to_json()), style_callback=self.fai_style)
        self.widget.add_layer(self.fai_layer)
        
    def update(self, state, rural_classification):
        self.replace_fai_layer(state)
        self.replace_rural_layer(state, rural_classification)
        self.widget.remove_control(self.layers_control)
        self.layers_control = leaflet.LayersControl()
        self.widget.add_control(self.layers_control)
        
    
    @staticmethod
    def poly_bounds(polygons):
        polygons = list(polygons)
        """Bounding box for `ipyleaflet.Map.fit_bounds()`."""
        xmin, ymin, xmax, ymax = shapely.geometry.MultiPolygon(polygons).bounds
        return [[ymin, xmax], [ymax, xmin]]

w_map = Map()

In [ ]:
w_map.widget

In [ ]:
data_states = data['CBSA']['econ shares'].index.get_level_values('STATE_ABBR').categories.tolist()
w_state = widgets.Dropdown(description='State', options=data_states)
w_rural_scheme = widgets.RadioButtons(description='Rurality', options=[('Core Based Statistical Area', 'CBSA'), ('Rural-Urban Continuum Codes', 'RUC'), ('Urban Influence Codes', 'UI')])

def update(change):
    w_shares.update(w_state.value, w_rural_scheme.value)
    w_map.update(w_state.value, w_rural_scheme.value)
    

w_state.observe(update, names='value')
w_rural_scheme.observe(update, names='value')

In [ ]:
w_state

In [ ]:
w_rural_scheme